In [1]:
from pyspark.sql import SparkSession

## TEST


In [2]:
spark = SparkSession.builder \
    .appName("Neo4j_PySpark_Demo") \
    .config(
        "spark.jars",
        r"D:\SPARK\spark-3.3.2-bin-hadoop3\jars\neo4j-connector-apache-spark_2.12-4.1.4_for_spark_3.jar"
    ) \
    .config("spark.neo4j.url", "bolt://127.0.0.1:7687") \
    .config("spark.neo4j.authentication.basic.username", "neo4j") \
    .config("spark.neo4j.authentication.basic.password", "12345678") \
    .config("spark.neo4j.database", "recommendations-50")\
    .config("spark.neo4j.database", "neo4j") \
    .getOrCreate()

In [4]:
query = "MATCH (n:Movie) RETURN n"
df = (
    spark.read.format("org.neo4j.spark.DataSource")
    .option("url", "bolt://127.0.0.1:7687")
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password", "12345678")
    .option("database", "recommendations-50")     
    .option("query", query)
    .load()
)

df.limit(25).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|n                                                                                                                                                                                                                                                                                                                                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------------

In [6]:
movies_df = (
    spark.read.format("org.neo4j.spark.DataSource")
        .option("url", "bolt://127.0.0.1:7687")
        .option("authentication.basic.username", "neo4j")
        .option("authentication.basic.password", "12345678")
        .option("database", "recommendations-50")         
        .option("labels", "Movie")                       
        .load()
)

movies_df.show(truncate=False)

+----+--------+----+----------------------------------+------+------------------------------+-------+---------+----------+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------------------------------+---------+----------+-------+--------------------------------+--------+
|<id>|<labels>|year|url                               |tmdbId|title                         |runtime|revenue  |released  |poster                                                                       |plot                                                                                                                                                                                                                   |movieId|languages                     |imdbVotes|imdbRati

In [8]:
data = [
    ("1", "Inception", 2010, 8.8, "Dreams within dreams"),
    ("2", "Interstellar", 2014, 8.6, "Exploration of space and time"),
    ("3", "Dune", 2021, 8.1, "The spice must flow")
]

columns = ["id", "title", "year", "rating", "tagline"]

df_movies = spark.createDataFrame(data, columns)
df_movies.show()

+---+------------+----+------+--------------------+
| id|       title|year|rating|             tagline|
+---+------------+----+------+--------------------+
|  1|   Inception|2010|   8.8|Dreams within dreams|
|  2|Interstellar|2014|   8.6|Exploration of sp...|
|  3|        Dune|2021|   8.1| The spice must flow|
+---+------------+----+------+--------------------+



In [10]:
(
    df_movies.write
    .format("org.neo4j.spark.DataSource")
    .option("url", "bolt://127.0.0.1:7687")
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password", "12345678")
    .option("database", "recommendations-50")
    .option("labels", ":Doan")
    .mode("append")     # 👈 chuyển sang append là được
    .save()
)


In [11]:
df_check = (
    spark.read.format("org.neo4j.spark.DataSource")
    .option("url", "bolt://127.0.0.1:7687")
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password", "12345678")
    .option("database", "recommendations-50")
    .option("labels", ":Doan")      # 👈 đọc các node có nhãn Doan
    .load()
)

df_check.show(10, truncate=False)


+-----+--------+----+------------+-----------------------------+------+---+
|<id> |<labels>|year|title       |tagline                      |rating|id |
+-----+--------+----+------------+-----------------------------+------+---+
|33880|[Doan]  |2014|Interstellar|Exploration of space and time|8.6   |2  |
|33881|[Doan]  |2021|Dune        |The spice must flow          |8.1   |3  |
|33882|[Doan]  |2010|Inception   |Dreams within dreams         |8.8   |1  |
+-----+--------+----+------------+-----------------------------+------+---+



In [13]:
spark.stop()

In [17]:
print("Spark version:", spark.version)
print("Scala version:", spark.sparkContext._gateway.jvm.scala.util.Properties.versionString())


Spark version: 3.3.2
Scala version: version 2.12.15
